In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.models as models
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader

## Load AlexNet model

In [2]:
alexnet = models.alexnet()
alexnet.classifier = nn.Sequential(*list(alexnet.classifier[:6]))
alexnet.load_state_dict(torch.load('Model/alexnet.pth'))

<All keys matched successfully>

## Load data

In [3]:
X = np.load("Data/X.npy")[2000:7000].astype(float)/255
y = np.load("Data/y.npy")[2000:7000]

In [4]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
 transforms.ToTensor(),
 transforms.Normalize(
 mean,std)])

def data_loader(X,y):
    d_ts = [transform(x) for x in X]
    d_ls = []
    for i in range(len(d_ts)):
        d_ls.append([d_ts[i],int(y[i])])
    loader = DataLoader(dataset = d_ls,batch_size=1,shuffle=False)
    return(loader)

torch.manual_seed(1)

data = data_loader(X,y)

del X,y

##### Extract features

In [5]:
features,labels = [],[]

alexnet.eval()
for feature,label in data:
  features.append(np.array(alexnet(feature.float()).detach().numpy()).reshape([4096]))
  labels.append(int(label))

features = np.array(features)
labels = np.array(labels)

del data

## Train SVM models for each classes

In [6]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [7]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=0)

##### Separate data by label

In [8]:
y1_train,y1_test = 0+(y_train==1), 0+(y_test==1)
y2_train,y2_test = 0+(y_train==2), 0+(y_test==2)

||uninfected|infected|background|
|------|------|------|------|
|train|1676|459|1765|
|test|734|31|735|

### Uninfected class

In [9]:
svm1 = svm.SVC(gamma='scale')
scores = cross_val_score(svm1, X_train, y1_train, cv=5)
print("Accuracy of uninfected class SVM: %.2f%% (+/- %.2f%%)" % (scores.mean()*100, scores.std() * 200))

Accuracy of uninfected class SVM: 88.89% (+/- 2.68%)


In [10]:
svm1.fit(X_train, y1_train)
print("Accuracy on test set: %.2f%% " % (svm1.score(X_test, y1_test)*100))

Accuracy on test set: 89.47% 


### Infected class

In [11]:
svm2 = svm.SVC(kernel='linear',class_weight='balanced')
scores = cross_val_score(svm2, X_train, y2_train, cv=5)
print("Accuracy of infected class SVM: %.2f%% (+/- %.2f%%)" % (scores.mean()*100, scores.std() * 200))

Accuracy of infected class SVM: 89.86% (+/- 3.70%)


In [12]:
svm2.fit(X_train, y2_train)
print("Accuracy on test set: %.2f%% " % (svm2.score(X_test, y2_test)*100))

Accuracy on test set: 88.87% 


### Save model

In [13]:
import pickle

pickle.dump(svm1, open('Model/svm1.sav', 'wb'))
pickle.dump(svm2, open('Model/svm2.sav', 'wb'))